In [11]:
import os
os.chdir("../")

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    best_model_path: Path

In [1]:
from objectDetection.constants import *
from objectDetection.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            training_data=Path(training.data_yaml_path),
            base_model_path=Path(prepare_base_model.base_model_path),
            best_model_path=Path(training.best_model_path),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGUMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [5]:
from ultralytics import YOLO
import shutil

In [6]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = YOLO(
            self.config.base_model_path
        )

    @staticmethod
    def save_model(path: Path, dest_path: Path):
        # model.save(path)
        shutil.copy(path, dest_path)


    def train(self):
        
        self.model.train(data=self.config.training_data, model="yolov8n.yaml",task="detect", \
                         epochs=self.config.params_epochs,workers=8,batch=self.config.params_batch_size,imgsz=self.config.params_image_size,\
                            project=self.config.root_dir,exist_ok=True)

        self.save_model(
            path=self.config.best_model_path,
            dest_path=self.config.trained_model_path
        )

In [13]:
import os
import yaml

def update_datasets_dir(path_to_script):
    # Determine the operating system
    is_windows = os.name == 'nt'
    
    # Define the YAML file path based on the operating system
    if is_windows:
        yaml_path = os.path.join(os.path.expanduser("~"), "AppData", "Roaming", "Ultralytics", "settings.yaml")
    else:
        yaml_path = os.path.join("/home/ajith", ".config", "Ultralytics", "settings.yaml")

    # Read the existing YAML file
    with open(yaml_path, 'r') as file:
        data = yaml.load(file, Loader=yaml.FullLoader)

    # Update the datasets_dir key with the current script path
    # data['datasets_dir'] = path_to_script
    data.pop('datasets_dir', None)

    # Write the updated YAML file
    with open(yaml_path, 'w') as file:
        yaml.dump(data, file)

update_datasets_dir(os.getcwd())


In [10]:
%pwd

'/home/ajith/projects/Multivideo_Objectdetection_MLOPS_Project/research'

In [16]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train()
    
except Exception as e:
    raise e


[2024-02-04 11:31:38,111: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-04 11:31:38,116: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-04 11:31:38,120: INFO: common: created directory at: artifacts]
[2024-02-04 11:31:38,122: INFO: common: created directory at: artifacts/training]
Ultralytics YOLOv8.1.9 🚀 Python-3.10.13 torch-2.2.0+cu121 CPU (12th Gen Intel Core(TM) i5-1235U)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=dataset.yaml, epochs=1, time=None, patience=50, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=artifacts/training, name=train, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=Fa

100%|██████████| 755k/755k [00:00<00:00, 5.03MB/s]

Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                


  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  8                  -1  1    460288  ultralytics.nn.modules.block.C2f             [256, 256, 1, True]           
  9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256, 256, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  1    148224  ultralytics.nn.modules.block.C2f             [384,

train: Scanning /home/ajith/projects/Multivideo_Objectdetection_MLOPS_Project/artifacts/data_ingestion/dataset/train/labels... 459 images, 0 backgrounds, 0 corrupt: 100%|██████████| 459/459 [00:01<00:00, 363.59it/s]


...

Plotting labels to artifacts/training/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 320 train, 320 val
Using 0 dataloader workers
Logging results to artifacts/training/train
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      1.649       3.14      1.058        188        320: 100%|██████████| 29/29 [01:05<00:00,  2.25s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:06<00:00,  2.16s/it]


                   all         90       1502      0.728     0.0763      0.106     0.0527

1 epochs completed in 0.021 hours.
Optimizer stripped from artifacts/training/train/weights/last.pt, 6.2MB
Optimizer stripped from artifacts/training/train/weights/best.pt, 6.2MB

Validating artifacts/training/train/weights/best.pt...
Ultralytics YOLOv8.1.9 🚀 Python-3.10.13 torch-2.2.0+cu121 CPU (12th Gen Intel Core(TM) i5-1235U)
Model summary (fused): 168 layers, 3006623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:05<00:00,  1.83s/it]


                   all         90       1502      0.728     0.0763      0.107     0.0527
               bicycle         90         30          1          0   0.000118   5.89e-05
                   bus         90          4          1          0    0.00417    0.00207
                   car         90       1030      0.642      0.382      0.459       0.21
             motorbike         90        400          0          0    0.00646    0.00208
                person         90         38          1          0     0.0636     0.0487
Speed: 0.6ms preprocess, 38.8ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to artifacts/training/train
